### ICECAPS-ACE Turbulent flux calculations
### 29-31 July 2019 melt event at Summit Station, Greenland.

Heather Guy, version: 2019-11-26

In [1]:
# Import useful things

import numpy as np      
import datetime as dt
from scipy.signal import medfilt, detrend, coherence, windows
import pandas as pd
import os

# Supress warnings for notebook ease of use
import warnings
warnings.filterwarnings("ignore")

In [2]:

### Data cleaning and parsing functions. 

def replace_outliers(var,sd):

    # Replace outliers with median filter. This function uses a window 
    # size of 11 data points for the median filter and defines outliers as
    # data that are greater than 3 standard deviations away from the median. 
    
    var=var.astype(float)
    jj = ~np.isnan(var) # Ignore nans
    temp = var[jj]
    mf = medfilt(temp,11)             # Get median filter.
    ii = np.abs(temp - mf) > 3*sd     # Get outliers.
    temp[ii] = mf[ii]                 # Replace these outliers with the median
    var_clean = var
    var_clean[jj] = temp              # Put back into orginal array
    
    # Interpolate over any Nans (default method = linear)
    
    var_clean = var_clean.interpolate()
    
    return var_clean


def clean_metek(m_in):

    # Clean Metek (3D sonic) data
    # Based on clean_metek.m by IAM, 25/7/2014
    # INPUT
    #  metek  - data strucure
    # OUTPUT
    #  metek  - data structure
    # wind compents and sonic temperature are cleaned up:
    # - the time series are filtered for outliers, these are replaced with  
    #   median filtered values
    # - missing values from error messages (NaNs) are then interpolated over
    
    m_out = m_in
    jj_z = ~np.isnan(m_in.z)         # Not nan indices - use to calculate standard deviation.
    jj_T = ~np.isnan(m_in['T'])
    m_sd_z = np.std(m_in.z[jj_z])    # standard deviation of vertical wind component. 
    m_sd_T = np.std(m_in['T'][jj_T]) # standard deviation of Temperature. 
    m_out['x']=replace_outliers(m_in['x'],m_sd_z)
    m_out['y']=replace_outliers(m_in['y'],m_sd_z)
    m_out['z']=replace_outliers(m_in['z'],m_sd_z)   
    m_out['T']=replace_outliers(m_in['T'],m_sd_T)

    return m_out


def Ts_sidewind_correction(Ts,u,v,w):

    # Do cross wind temperature correction
    # Adapted from Ts_sidewind_correction.m by IMB July 2007
    # Correct contamination of sonic temperature measurement for lengthening of
    # sonic path by sidewind. 
    # INPUTS
    #  Ts    : sonic temperature (K)
    #  u,v,w : wind components in sonic measurement frame (before any rotations,
    #          motion correction, etc) (m/s)
    # OUTPUT
    #  T     : corrected temperature (K)
    #
    #     Correction follows van Dijk et al. 2004: The
    #     principles of surface flux physics: theory, practice and description of
    #     the ECPACK library (www.met.wau.nl/projects/jep). See also: Liu et al.
    #     2001: BLM, 100, 459-468, and Schotanus et al. 1983: BLM, 26, 81-93.

    vn2 = (3/4)*(u**2 + v**2) + 0.5*w**2
    T = Ts + vn2/403
    return T


def rotate_to_run(m,avp):
    # Correct tilt and align with horizontal streamline over a single run
    # Adapted from rotate_to_run.m by IMB July 2006
    #% references:
    #%  Wilczak et al. 2001: sonic anemometer tilt corrections. BLM, 99, 127-150
    #%  (but beware typos in equations)
    #%  Kaimal & Finnigan, 1994: Atmospheric Boundary Layer Flows: Their
    #%  Structure and Measurement. Oxford University Press
    #%  van Dijk et al. 2004: The principles of surface flux physics: theory,
    #%  practice and description of the ECPACK library
    #%  www.met.wau.nl/projects/jep
    
    # INPUTS
    #  m     : Unrotated metek data structure.
    #  avp   : Length of a single run (minutes). i.e. averaging period. 
    # OUTPUT
    #  m_out     : Wind components in streamline oriented reference frame
    
    # First rotate to align x-axis with mean wind direction in sonic's
    # reference frame
    
    m_g = m.groupby(pd.Grouper(freq='%sMin'%avp))     # Split into single runs (of length avp minutes) 
    m_out = pd.DataFrame(columns=['x','y','z','T','u','v','w','theta','phi'])
    
    # Loop through each run to perform correction: 
    for group in m_g:
        m = group[1]
        
        # First rotate to align x-axis with mean wind direction in sonic's
        # reference frame
        
        theta=np.arctan2(np.mean(m['y']),np.mean(m['x']))
        u1 = m['x']*np.cos(theta) + m['y']*np.sin(theta)
        v1 = -m['x']*np.sin(theta) + m['y']*np.cos(theta)
        w1 = m['z']

        # Next rotate u and w so that x-axis lies along mean streamline and 
        # mean(w) is zero
        
        phi = np.arctan2(np.mean(w1),np.mean(u1))
        m['u'] = u1*np.cos(phi) + w1*np.sin(phi)
        m['v'] = v1
        m['w']=  -u1*np.sin(phi) + w1*np.cos(phi)
        
        # Theta is angle of rotation um-to-vm (anticlockwise or righthanded)
        # to aign u with mean wind (degrees)
        
        m['theta'] = theta*180/np.pi

        # phi is tilt angle (+ve tilts x-axis upwards) to align x-axis with
        # mean streamline and force <w>=0
        
        m['phi'] = phi*180/np.pi
        m_out = m_out.append(m)
    return m_out


def eddyflux(x,y):

    # Function to calculate instantaneous flux 
    # x is time series of vertical velocity
    # y is parameter of interest.
    
    xprime = detrend(x)
    yprime = detrend(y)
    flux = np.mean(xprime * yprime) # Instantaneous flux
    std = np.std(xprime * yprime)   # Standard deviation of instantaneous flux

    return flux,std


def licor_mmr(T,P,Nconc):
    
    # Calculate mass mixing ratio from LiCOR. 
    # Requires P from Licor, Nconc from Licor, T from HMP
    # Converts Nconc (molar number concentration) to Mass mixing ration q (kg/kg)
    
    Ma = 28.96        # Molar mass of dry air (g/mol)
    Mh = 18.01528     # Molar mass of H2O (g/mol)
    R = 8.314         # Universal gas constant (J/K/mol)
    
    # Join and interpolate temperature (linear interpolation)
    temp_df = pd.concat([T, P, Nconc],axis=1,keys=['T','P','Nconc'])
    temp_df['T'] = temp_df['T'].interpolate(limit=200)
    
    # Calculate q
    mass_conc_air = (Ma*temp_df['P'])/(R*temp_df['T']) / 1000    # kg air/ m3
    mass_conc_h2o = (Mh * temp_df['Nconc']) / 1000               # kg water / m3
    q = mass_conc_h2o / mass_conc_air                            # Mass mixing ratio   
    q = q.rename('q')
    
    return q


def rho_m(T,P,q):

    # Estimate air density using time varying T, P and q.
    # Assume ideal gas behaviour
    # P: static pressure (mb) from licor
    # T: air temperature (K) from HMP1
    # q, mass mixing ratio (kg/kg) from licor.  

    Rd = 287 # J K-1 kg-1, gas constant for dry air.     

    temp_df = pd.concat([T, P, q],axis=1,keys=['T','P','q'])
    temp_df['T'] = temp_df['T'].interpolate(limit=200) # Interpolate temperature
    
    rho_m =  1/ ((Rd * temp_df['T']) *(1+0.61*temp_df['q'])) * temp_df['P']
    
    return rho_m


def get_windd(uwind,vwind):
    
    # Converts u and v wind components into meteorological degrees.
    # meteorologica degrees = clockwise, north =0, direction wind is coming from.
    # uwind = wind in u direction (north)
    # vwind = wind in v direction (east)
    
    dir = 180 + ((np.arctan2(uwind/(np.sqrt(uwind**2 + vwind**2)), vwind/(np.sqrt(uwind**2 + vwind**2)))) * 180/np.pi) # degrees
    
    return dir


def deg_rot(orig,diff):
    
    # Adds a fixed number of degrees to a wind direction. 
    # To account for sonics not being oriented to north. 
    # orig = list or series of wind directions in degrees. 
    # diff = number of degrees clockwise to rotate (can be negative).
    
    new = orig
    new = new + diff
    new[new<0] = 360 + (orig[new<0] + diff)
    new[new>360] = diff - (360 - orig[new>360])
    new[new==360] = 0

    return new


def stationarity(x, y):

    # Test for stationarity: 
    # Divide averaging period into smaller segments say 5 minutes each. 
    # Calculate the covariance of each interval, then calculate the mean. 
    # Also calculate the covariance of the whole interval. 
    # If there is a difference of less than 30% between the covariances, 
    # then the measurement is considered to be stationary. [foken and wichura 1995]
    
    # 30% is a strict condition for fundametal science
    # Use < 75 % to compromise 
    
    # x,y = variables to test.
    
    # Return: 
    # fs: list of stationarity parameter from each period, defined above. 
    # pc: list of covariance between variables for each period. 
    # rc: list of 5 min rolling covariances between variables for each period. 

    xw=detrend(x)
    yw=detrend(y)
 
    df = pd.DataFrame(list(zip(xw, yw)), 
               columns =['x', 'y'])

    # Calculate covariance over the entire period. 
        
    p_cov = df['x'].cov(df['y'])
        
    # Calculate a rolling covariance, window size = 5 mins (3000 data points).
        
    rol_cov = df.rolling(3000).cov().unstack()['x']['y']

    # Calculate stationarity parameter. 
    
    mean_segs = np.mean(rol_cov)
    fs = np.abs((mean_segs - p_cov) / p_cov)
        
    return fs, p_cov, rol_cov


In [3]:

# Function to calculate latent heat flux

def lhf(w,q,rho,avp):
    
    # Calculate latent heat flux by eddy covariance. 
    # Calculate the covariance between the anomalies in the vertical wind (w') and H2O mass mixing ratio (q')
    # to determine the turbulent latent heat flux according to the following equation:
    # lhf = rho * L * mean(w'*q')
    # rho = density of air       
    # L = Latent heat of vaporisation
    
    # To calculate mean(w'*q'), use eddyflux function

    # lhf represents the loss of energy by the
    # surface due to evaporation/ sublimation.
    # It is positive when directed away from the surface.
    
    LHF=[]
    stds=[]
    
    L = 2264705.0     # Latent heat of vaporistaion of water (j/kg)
  
    # Split data into groups - seperate group for each averaging period
    
    w_g = w.groupby(pd.Grouper(freq='%sMin'%avp))  
    q_g = q.groupby(pd.Grouper(freq='%sMin'%avp)) 
    rho_g = rho.groupby(pd.Grouper(freq='%sMin'%avp))     
    keys = list(w_g.groups.keys())
    
    # For each averaging period, calculate the LHF
    
    for i in range(0,len(keys)):
        w1 =w_g.get_group(keys[i])
        q1 =q_g.get_group(keys[i])
        rho1 =rho_g.get_group(keys[i])
        
        # Mean air density for this averaging period
        rho_mean = rho1.mean()
    
        # Join data frames so they're the same length, remove nans
        join = pd.concat([q1, w1], axis=1, sort=False, names=['q','w'])
        join = join.dropna()
        
        # Check there's at least 60% of averaging period data
        if len(join) >= 0.6 * (avp*60) * 10:
            # Test for stationarity - Not yet implemented
            # fs, pc, rc = stationarity(join['w'],join['q'])
            # if fs < 0.75: 
            
            flux,std = eddyflux(join['w'],join['q'])
            LHF.append(L * rho_mean * flux)
            stds.append(std)
            
            #else:
            #    print('Failed stationarity test for %s, LHF'%str(keys[i]))
            #    print(fs)
            #    LHF.append(np.nan)
            #    stds.append(np.nan)
            
        else:
            print('Not enough good data for %s, LHF'%str(keys[i]))
            LHF.append(np.nan)
            stds.append(np.nan)
            
    return LHF, stds

In [9]:
# Function to calculate sensible heat flux: 

def shf(w,t,rho,avp):

    # Calculate sensible heat flux. 
    # The EC method (e.g., Oke, 1987) calculates the covariance
    # between the anomalies in the vertical wind (w') and temperature (T')
    # to determine the turbulent sensible heat flux according to the following equation:
    # SH = rho * cp * mean(w'*T')
    # rho = density of air       
    # cp = heat capacity of air

    # To calculate mean(w'*T'), use eddyflux function

    # shf represents the loss of energy from the
    # surface by heat transfer to the atmosphere.
    # It is positive when directed away from the surface.

    SHF=[]
    stds=[]
    
    cp = 1003         # Specific heat capacity of dry air at constant pressure (j/kg/K) - at 250K

    # Split data into groups - seperate group for each averaging period
    
    w_g = w.groupby(pd.Grouper(freq='%sMin'%avp))  
    t_g = t.groupby(pd.Grouper(freq='%sMin'%avp)) 
    rho_g = rho.groupby(pd.Grouper(freq='%sMin'%avp))     
    keys = list(w_g.groups.keys())
    
    # For each averaging period, calculate the SHF
    
    for i in range(0,len(keys)):
        w1 =w_g.get_group(keys[i])
        t1 =t_g.get_group(keys[i])
        rho1 =rho_g.get_group(keys[i])
        
        # Mean air density for this averaging period
        rho_mean = rho1.mean()
        
        # Join data frames so they're the same length, remove nans
        join = pd.concat([t1, w1], axis=1, sort=False, names=['T','w'])
        join = join.dropna()

        # Check there's at least 60% of averaging period data
        if len(join) >= 0.6 * (avp*60) * 10:
            # Test for stationarity - NOT YET IMPLEMENTED
            # fs, pc, rc = stationarity(join['w'],join['T'])
            # if fs < 0.75: 
            flux,std = eddyflux(join['w'],join['T'])
            SHF.append(rho_mean * cp * flux) # W/m2
            stds.append(std)
            #else:
            #    print('Failed stationarity test for %s, SHF'%str(keys[i]))
            #    print(fs)
            #    SHF.append(np.nan)
            #    stds.append(np.nan)
        else:
            #print(len(join))
            print('Not enough good data for %s, SHF'%str(keys[i]))
            SHF.append(np.nan)
            stds.append(np.nan)            
                
    return SHF, stds


In [10]:

# Actually Calculate the fluxes here. 

# Data location:
d_loc = '../data/TurbulentFluxes/'

# Start and stop date:
start = dt.datetime(2019,7,29,0,0)
stop = dt.datetime(2019,7,31,0,0)
# Days to loop through
days = pd.date_range(start,stop,freq='1D')

# 1) Initialize data frame

flux_pro_15 = pd.DataFrame(columns=['Dates', 'SHF', 'LHF']) # 15 min averaging period
flux_pro_30 = pd.DataFrame(columns=['Dates', 'SHF', 'LHF']) # 30 min averaging period

# 2) Loop through each day

for day in days:
    day_str = str(day.date()) 
    print(day_str)

# 3) Get the 3D sonic data

    if os.path.isfile(d_loc+'3D_sonic/metek1_%s'%day_str):
        m1_orig = pd.read_csv(d_loc+'3D_sonic/metek1_%s'%day_str, index_col=0, parse_dates=[0])
        if m1_orig.empty:
            print('Error: File empty, '+day_str)
            continue
    else:
        print('Error: File empty, '+day_str)
        continue
        
# 4) Get licor data

    if os.path.isfile(d_loc+'LiCOR/licor_%s'%day_str):
        licor = pd.read_csv(d_loc+'LiCOR/licor_%s'%day_str, index_col=0, parse_dates=[0])
    else:
        print('Error: File empty, '+day_str)
        continue
        
# 5) Get HMP 2m T data

    if os.path.isfile(d_loc+'HMP1/HMP1_%s'%day_str):
        HMP1 = pd.read_csv(d_loc+'HMP1/HMP1_%s'%day_str, index_col=0, parse_dates=[0])
        # Crop to date, time
        HMP1 = HMP1[day:day+pd.Timedelta(hours=24)] 
    else:
        print('Error: File empty, '+day_str)
        continue
        
# 6) Clean metek data 

    m1 = clean_metek(m1_orig)

# 7) Implement cross-wind temperature correction

    m1['T'] = Ts_sidewind_correction(m1['T'],m1['x'],m1['y'],m1['z'])

# 8) Rotate to average streamline for each 15 minutes. 

    m_rot = rotate_to_run(m1,15)
    
# 9) Mask flow distortion from tower or camp (wind direction between 17 and 90 degrees)

    m_rot['wdir_arb'] = get_windd(m_rot['x'],m_rot['y']) # wind direction in sonic coords
    m_rot['wdir_cor'] = deg_rot(m_rot['wdir_arb'],-45)   # wind direction corrected for sonic orientation.
    
    distortion_mask = m_rot['wdir_cor'].between(17, 90, inclusive = True)
    m_rot['w'][distortion_mask] = np.nan                 # Mask vertical wind fluctuations where flow may be distorted. 
    
# 9) Calculate mass mixing ratio from licor and estimate air density

    Ta = HMP1['Ta'] + 273.15   # 2m air temperature,  K
    P = licor['P']             # 2m air pressure form licor, Pa
    Nconc = licor['H2OD']      # H2O number concentration from licor, mol/m3
 
    q = licor_mmr(Ta,P,Nconc)  # H2O mass mixing ratio, kg/kg  
    rho = rho_m(Ta,P,q)        # Air density estimation, kg/m3

# 10) Calculate sensible heat flux. 

    w = m_rot['w']             # 2m 3D sonic vertical velocity (corrected)
    T = m_rot['T']             # 2m 3D sonic virtual temperature (corrected)

    SHF_15,shf_std=shf(w,T,rho,15) # 15 minute averaging period
    SHF_30,shf_std=shf(w,T,rho,30) # 30 minute averaging period

# 11) Calculate latent heat flux. 

    LHF_15,lhf_std=lhf(w,q,rho,15) # 15 minute averaging period
    LHF_30,lhf_std=lhf(w,q,rho,30) # 30 minute averaging period
        
# 12) Change sign convention so that fluxes are directed into the surface

    SHF_15 = -np.asarray(SHF_15)
    LHF_15 = -np.asarray(LHF_15)
    
    SHF_30 = -np.asarray(SHF_30)
    LHF_30 = -np.asarray(LHF_30)

# 13) Build output dataframe of sensible and latent heat fluxes

    flux_temp_15 = pd.DataFrame(columns=['Dates', 'SHF', 'LHF'])
    flux_temp_30 = pd.DataFrame(columns=['Dates', 'SHF', 'LHF'])

    flux_temp_15['Dates']= pd.date_range(day, day+pd.Timedelta(hours=24),freq='15 min')[0:-1]
    flux_temp_15['SHF']=SHF_15
    flux_temp_15['LHF']=LHF_15
    flux_temp_15 = flux_temp_15.set_index('Dates')
    
    flux_temp_30['Dates']= pd.date_range(day, day+pd.Timedelta(hours=24),freq='30 min')[0:-1]
    flux_temp_30['SHF']=SHF_30
    flux_temp_30['LHF']=LHF_30
    flux_temp_30 = flux_temp_30.set_index('Dates')

    
    flux_pro_15 = flux_pro_15.append(flux_temp_15)
    flux_pro_30 = flux_pro_30.append(flux_temp_30)
    
flux_pro_15.set_index('Dates')
flux_pro_15 = flux_pro_15.sort_values('Dates')
flux_pro_15.index = pd.DatetimeIndex(flux_pro_15.index)
del flux_pro_15['Dates']

flux_pro_30.set_index('Dates')
flux_pro_30 = flux_pro_30.sort_values('Dates')
flux_pro_30.index = pd.DatetimeIndex(flux_pro_30.index)
del flux_pro_30['Dates']


2019-07-29
Not enough good data for 2019-07-29 05:30:00, SHF
Not enough good data for 2019-07-29 05:45:00, SHF
Not enough good data for 2019-07-29 06:00:00, SHF
Not enough good data for 2019-07-29 06:15:00, SHF
Not enough good data for 2019-07-29 06:45:00, SHF
Not enough good data for 2019-07-29 05:30:00, SHF
Not enough good data for 2019-07-29 06:00:00, SHF
Not enough good data for 2019-07-29 06:30:00, SHF
Not enough good data for 2019-07-29 05:30:00, LHF
Not enough good data for 2019-07-29 05:45:00, LHF
Not enough good data for 2019-07-29 06:00:00, LHF
Not enough good data for 2019-07-29 06:15:00, LHF
Not enough good data for 2019-07-29 06:45:00, LHF
Not enough good data for 2019-07-29 17:30:00, LHF
Not enough good data for 2019-07-29 18:00:00, LHF
Not enough good data for 2019-07-29 18:30:00, LHF
Not enough good data for 2019-07-29 18:45:00, LHF
Not enough good data for 2019-07-29 19:00:00, LHF
Not enough good data for 2019-07-29 19:15:00, LHF
Not enough good data for 2019-07-29 19:

In [11]:

# Save data if required.

flux_pro_15.to_csv(d_loc+'TurbulentFluxes_15min.csv')
flux_pro_30.to_csv(d_loc+'TurbulentFluxes_30min.csv')


In [12]:

# Plot sensible and latent heat fluxes

# get data
flux_pro_15 = pd.read_csv('../data/TurbulentFluxes/TurbulentFluxes_15min.csv',parse_dates=[0],index_col=[0])
flux_pro_30 = pd.read_csv('../data/TurbulentFluxes/TurbulentFluxes_30min.csv',parse_dates=[0],index_col=[0])

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3
from bokeh.plotting import figure
from bokeh.models import Title
from bokeh.models import Span

output_notebook()

# Horizontal line
hline = Span(location=0, dimension='width', line_color='orange', line_width=1)

p1=figure(plot_width=1000,
       plot_height=400,
       y_axis_label='Heat Flux W/m2',
       x_axis_type='datetime',
       y_range=(-30, 30))

p1.renderers.extend([hline])
p1.line(flux_pro_15.index,flux_pro_15['SHF'],color='blue',legend='Sensible heat flux')
p1.line(flux_pro_15.index,flux_pro_15['LHF'],color='red',legend='Latent heat flux')

p1.add_layout(Title(text="Positive fluxes are directed into the surface"), 'above')
p1.add_layout(Title(text="Averaging period = 15 minutes"), 'above')



p2=figure(plot_width=1000,
       plot_height=400,
       y_axis_label='Heat Flux W/m2',
       x_axis_type='datetime',
       y_range=(-30, 30))

p2.renderers.extend([hline])
p2.line(flux_pro_30.index,flux_pro_30['SHF'],color='blue',legend='Sensible heat flux')
p2.line(flux_pro_30.index,flux_pro_30['LHF'],color='red',legend='Latent heat flux')

p2.add_layout(Title(text="Positive fluxes are directed into the surface"), 'above')
p2.add_layout(Title(text="Averaging period = 30 minutes"), 'above')


show(p1)
show(p2)

Loading BokehJS ...

In [14]:
# friction velocity (u*) ? - Coming soon

In [25]:
# the Obukhov length (L) ? - Coming soon